In [1]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

/home/dluser1/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.6.0


# Puzzle description

Question: only cacluation for the existing element for element-wise multiplication, how to implement in tensorflow?

$
\begin{pmatrix}
1 &  & & 2\\ 
& 3 & & 4 \\     
5 & 6 & & 
\end{pmatrix}
$
*
$
\begin{pmatrix}
0.1 &  & & 0.2\\ 
& 0.3 &0.01 & 0.4 \\ 
0.5 & 0.6 & & 
\end{pmatrix}
$

## 1, Sparse representation

In [2]:
sparse1 = tf.SparseTensor(indices=[[0, 0], [0, 3], [1, 1], [1, 3], [2, 0], [2, 1]], 
                values=np.asarray([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=np.float32), dense_shape=[3, 4])
sparse2 = tf.SparseTensor(indices=[[0, 0], [0, 3], [1, 1], [1, 2], [1, 3], [2, 0], [2, 1]], 
                values=np.asarray([0.1, 0.2, 0.3, 0.01, 0.4, 0.5, 0.6], dtype=np.float32), dense_shape=[3, 4])

In [3]:
t1 = tf.cast(tf.cast(sparse1, tf.bool), tf.int32)
t2 = tf.cast(tf.cast(sparse2, tf.bool), tf.int32)
sparse_mask = tf.sets.set_intersection(t1, t2)

In [4]:
with tf.Session() as sess:
    print(sess.run(t1))
    print(sess.run(t2))
    print(sess.run(sparse_mask))

SparseTensorValue(indices=array([[0, 0],
       [0, 3],
       [1, 1],
       [1, 3],
       [2, 0],
       [2, 1]]), values=array([1, 1, 1, 1, 1, 1], dtype=int32), dense_shape=array([3, 4]))
SparseTensorValue(indices=array([[0, 0],
       [0, 3],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1]]), values=array([1, 1, 1, 1, 1, 1, 1], dtype=int32), dense_shape=array([3, 4]))
SparseTensorValue(indices=array([[0, 0],
       [1, 0],
       [2, 0]]), values=array([1, 1, 1], dtype=int32), dense_shape=array([3, 1]))


In [5]:
import collections

a = collections.OrderedDict([
      ((0, 0), 1),
      ((0, 3), 1),
      ((1, 1), 1),
      ((1, 3), 1),
      ((2, 0), 1),
      ((2, 1), 1)
  ])
a = tf.SparseTensor(list(a.keys()), list(a.values()), dense_shape=[3, 4])

b = collections.OrderedDict([
      ((0, 0), 1),
      ((0, 3), 1),
      ((1, 1), 1),
      ((1, 2), 1),
      ((1, 3), 1),
      ((2, 0), 1),
      ((2, 1), 1)
  ])
b = tf.SparseTensor(list(b.keys()), list(b.values()), dense_shape=[3, 4])

with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(tf.contrib.metrics.set_intersection(a, b)))

SparseTensorValue(indices=array([[0, 0],
       [0, 3],
       [1, 1],
       [1, 3],
       [2, 0],
       [2, 1]]), values=array([1, 1, 1, 1, 1, 1], dtype=int32), dense_shape=array([3, 4]))
SparseTensorValue(indices=array([[0, 0],
       [0, 3],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1]]), values=array([1, 1, 1, 1, 1, 1, 1], dtype=int32), dense_shape=array([3, 4]))
SparseTensorValue(indices=array([[0, 0],
       [1, 0],
       [2, 0]]), values=array([1, 1, 1], dtype=int32), dense_shape=array([3, 1]))


## 2, Dense representation

In [6]:
import collections

a = collections.OrderedDict([
      ((0, 0, 0), 1),
      ((0, 0, 1), 2),
      ((0, 1, 0), 3),
      ((1, 0, 0), 4),
      ((1, 1, 0), 5),
      ((1, 1, 1), 6),
  ])
a = tf.SparseTensor(list(a.keys()), list(a.values()), dense_shape=[2,2,4])

b = collections.OrderedDict([
      ((0, 0, 0), 1),
      ((1, 0, 0), 4),
      ((1, 1, 0), 5),
      ((1, 1, 1), 6),
      ((1, 1, 2), 7),
      ((1, 1, 3), 8),
  ])
b = tf.SparseTensor(list(b.keys()), list(b.values()), dense_shape=[2, 2, 4])

with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(tf.sets.set_intersection(a, b)))

SparseTensorValue(indices=array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 1]]), values=array([1, 2, 3, 4, 5, 6], dtype=int32), dense_shape=array([2, 2, 4]))
SparseTensorValue(indices=array([[0, 0, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 1, 3]]), values=array([1, 4, 5, 6, 7, 8], dtype=int32), dense_shape=array([2, 2, 4]))
SparseTensorValue(indices=array([[0, 0, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 1]]), values=array([1, 4, 5, 6], dtype=int32), dense_shape=array([2, 2, 2]))
